In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import LSTM,Dense,Input,Embedding,Dropout
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import pickle
from os.path import split
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/model/news_data_Target.csv')
data = data[['Target','content_title','content_summary']].dropna()
data = pd.concat([data,pd.get_dummies(data['Target'])],axis=1).drop('Target',axis=1)
data['artical'] = data['content_title'] + ' ' + data['content_summary']
labels =  np.array(data['Bullish'],dtype=np.int32) # 1=positive, 0=negative
imdb_train, imdb_test = train_test_split(data, test_size=0.20, random_state=42)
x_train = imdb_train['artical']
y_train = imdb_train['Bullish']
x_test = imdb_test['artical']
y_test = imdb_test['Bullish']

In [ ]:
#Load the tokenizer
with open(f'/content/drive/MyDrive/model/tokenizer.pickle', 'rb') as handle:
    tk = pickle.load(handle)


In [ ]:
vocab_size = len(tk.word_index)
max_len = 300

In [ ]:
seq_train = tk.texts_to_sequences(x_train)
seq_test = tk.texts_to_sequences(x_test)

In [ ]:
seq_train_matrix = sequence.pad_sequences(seq_train,maxlen = max_len)
seq_test_matrix = sequence.pad_sequences(seq_test,maxlen = max_len)

In [ ]:
embedding_matrix = np.load(f'/content/drive/MyDrive/model/embedding_matrix.npy')

In [ ]:
model_path = f'/content/drive/MyDrive/model/model.weights.h5' # Changed the path to load weights

try:
  # Load the model architecture (you'll need the .json file for this)
  # If you have the .json file in a different location, update the path accordingly
  json_model_path = f'/content/drive/MyDrive/model/model.json' # Assumed the .json is here
  with open(json_model_path, 'r') as json_file:
    loaded_model_json = json_file.read()

  model = keras.models.model_from_json(loaded_model_json)

  # Now load the weights into the model
  model.load_weights(model_path)
  print("Model weights loaded successfully.")
  # Compile the model (necessary for prediction)
  model.compile(optimizer='adam', # Replace with your actual optimizer and loss function
                loss='mse', # Replace with your actual loss function
                metrics=['accuracy']) # Replace with your actual metrics

except OSError as e:
  print(f"Error loading model: {e}")
except FileNotFoundError as e:
    print(f"File not found: {e}")
except Exception as e:
  print(f"An unexpected error occurred: {e}")

Model weights loaded successfully.


In [ ]:
imdb_test['content_title'].reset_index(drop=True, inplace=True)
imdb_test['content_summary'].reset_index(drop=True, inplace=True)
for i in range(20):
  print("Title : ",imdb_test['content_title'][i])
  print("Summary :",imdb_test['content_summary'][i])
  if(model.predict(seq_test_matrix[i].reshape(1,-1)) > 0.5):
    print('Bullish\n')
  else:
    print('Bearish\n')

Title :  Better Monthly Dividend Stock: Realty Income vs. Agree Realty
Summary : At the core, Agree Realty (NYSE: ADC) and Realty Income (NYSE: O) are very similar real estate investment trusts (REITs).  Here's why some investors will like Agree, while others will favor Realty Income.  It all boils down to the growth these two monthly dividend payers can offer to shareholders.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 363ms/step
Bearish

Title :  WEC Energy Group Inc (WEC) Q4 2024 Earnings Call Highlights: Record Capital Plan and Dividend ...
Summary : WEC Energy Group Inc (WEC) reports robust earnings growth and unveils a historic $28 billion capital plan, while navigating weather impacts and regulatory hurdles.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
Bearish

Title :  Here's Why Rockwell Automation Stock Surged This Week
Summary : Shares in Rockwell Automation (NYSE: ROK) rose by 11.3% in the week to Friday morning.  Investors usually follow orders growth at Rockwell because it indicates the industr

In [ ]:
content_title = input("Enter the content title: ")
content_summary = input("Enter the content summary: ")

Enter the content title: Looking for Dividend Stocks to Buy in February? Consider These 3 Dow Jones Components.
Enter the content summary: ThThe Dow JonThe Dow Jones Industrial Average (DJINDICES: ^DJI) is chock-full of industry-leading dividend-paying companies -- making it a great place to look for stocks to boost your passive income stream.  In addition to paying dividends, some Dow stocks -- like 3M (NYSE: MMM), Honeywell International (NASDAQ: HON), and Cisco Systems (NASDAQ: CSCO) -- are also a good value relative to their earnings.  Three Motley Fool contributors were asked to offer up reasons why all three of these Dow stocks could be worth buying in February.es Industrial Average (DJINDICES: ^DJI) is chock-full of industry-leading dividend-paying companies -- making it a great place to look for stocks to boost your passive income stream.  In addition to paying dividends, some Dow stocks -- like 3M (NYSE: MMM), Honeywell International (NASDAQ: HON), and Cisco Systems (NASDAQ: C

In [ ]:

artical = content_title + ' ' + content_summary
seq_test = tk.texts_to_sequences([artical])
seq_test_matrix_ = sequence.pad_sequences(seq_test,maxlen = max_len)

In [ ]:
print("Title : ",content_title)
print("Summary :",content_summary)
if(model.predict(seq_test_matrix_.reshape(1,-1)) > 0.5):
  print('Bullish')
else:
  print('Bearish')

Title :  Looking for Dividend Stocks to Buy in February? Consider These 3 Dow Jones Components.
Summary : ThThe Dow JonThe Dow Jones Industrial Average (DJINDICES: ^DJI) is chock-full of industry-leading dividend-paying companies -- making it a great place to look for stocks to boost your passive income stream.  In addition to paying dividends, some Dow stocks -- like 3M (NYSE: MMM), Honeywell International (NASDAQ: HON), and Cisco Systems (NASDAQ: CSCO) -- are also a good value relative to their earnings.  Three Motley Fool contributors were asked to offer up reasons why all three of these Dow stocks could be worth buying in February.es Industrial Average (DJINDICES: ^DJI) is chock-full of industry-leading dividend-paying companies -- making it a great place to look for stocks to boost your passive income stream.  In addition to paying dividends, some Dow stocks -- like 3M (NYSE: MMM), Honeywell International (NASDAQ: HON), and Cisco Systems (NASDAQ: CSCO) -- are also a good value rel